In [ ]:
# This defines a function that plots the X for three classes 
# and updates the plot
from IPython.display import clear_output
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
import collections
%matplotlib inline

def live_plot(X, y, figsize=(7,5), title=''):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    plt.scatter(X[y==0].T[0], X[y==0].T[1], color='red')
    plt.scatter(X[y==1].T[0], X[y==1].T[1], color='blue')
    plt.scatter(X[y==2].T[0], X[y==2].T[1], color='green')
    plt.title(title)
    plt.show();

In [ ]:
# Import pytorch - 
# If you are in an Anaconda environment, you can use: 
# conda install pytorch torchvision -c pytorch
import numpy as np
import torch
import pandas as pd
from sklearn.datasets import make_blobs, make_circles

In [ ]:
# Make a circle data set 
X, y = make_circles(random_state=0, noise=0.01)

In [ ]:
# And plot it. 
plt.scatter(X[y==0,0], X[y==0,1], color='red')
plt.scatter(X[y==1,0], X[y==1,1], color='blue')
plt.show()

In [ ]:
# In pytorch, a custom network is defined as a 
# class. If you not familiar with object-oriented programming 
# and classes in Python read for example: 
# https://www.python-course.eu/python3_inheritance.php
# A Pytorch module has at least a constructor (__init__)
# and a function that defines how information is passed through the 
# network in forward direction. 
class OneLayerModel(torch.nn.Module):
    
    def __init__(self, input_size, num_classes):
        
        super().__init__()
        
        # Neural Network Architecture
        self.dense1 = torch.nn.Linear(in_features=num_features, out_features=num_classes)
        self.activation = torch.nn.LogSigmoid()
    
    def forward(self, X):
        X = self.dense1(X)
        X = self.activation(X)
        return X

In [ ]:
# Now we can use the class to make an instance of our network 
# and let it learn the circle problem 
num_features = X.shape[1]
num_classes = 2
model = OneLayerModel(num_features, num_classes) # Generate an instance of our network

In [ ]:
# Instead of numpy arrays, Pytorch uses torch.Tensor 
# Objects. They behave quite similar to numpy arrays, with the 
# difference that (Y=X) creates a copy. 
# Importantly, the allow you to automatically get a derivative 
# after some operations 
x1 = torch.tensor([0.0,0.0,1.0,2.0,1.0], requires_grad=True)
out = x1.pow(2).sum()
out.backward()
x1.grad

In [ ]:
# This also works when two elements are involved 
x1 = torch.tensor([0.0,0.0,1.0,2.0,1.0], requires_grad=True)
x2 = torch.tensor([1.0,1.0,0.0,0.0,0.0], requires_grad=True)
out = (x1*x2).sum()
out.backward()
x1.grad

In [ ]:
# This even works when the 
# operations are chained. 
x1 = torch.tensor([0.0,0.0,1.0,2.0,1.0], requires_grad=True)
# x2 = torch.tensor([1.0,1.0,0.0,0.0,0.0], requires_grad=True)
x2 = x1.pow(2)
out = x2.sum()
out.backward()
x1.grad

In [ ]:
# Now we are making our input and desired output pytorch tensors without derivative 
Xt = torch.FloatTensor(X)
yt = torch.LongTensor(y)

In [ ]:
# Now we can get a forward pass 
y_pred = model.forward(Xt)


In [ ]:
# The loss function is also an object with attached derivative 
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(input=y_pred, target=yt)
loss

In [ ]:
# Optimization is implemented in a torch.optim object 
# here we are using stochastic gradient decent (SGD)
# model.parameters() is a link to the parameters of the model, 
# Which allows the optimize to change it. 
# input argument lr is learning rate...  
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

In [ ]:
# Now put these together and optimise 
max_iter = 50000

for i in range(max_iter):
    # Intialize the gradient 
    optimizer.zero_grad()
    # Get current 
    y_pred = model.forward(Xt) # Get a forward pass with gradient 
    loss = criterion(input=y_pred, target=yt) # Caluculate the loss  
    loss.backward() # propagate the derivative backwards 
    optimizer.step() # Take one updating step
    if i % 100 == 0:
        with torch.no_grad():
            live_plot(y_pred.detach().numpy(), y, title=f"Loss at epoch {i}: {loss.item():.4f}")

In [ ]:
# To get the accuracy, we can for computational efficiency run it without gradients 
with torch.no_grad():
    y_pred = model.forward(Xt)
    y_pred = torch.argmax(y_pred, dim=1).detach().numpy()
    print(f"Classifier Accuracy: {accuracy_score(y_true=y, y_pred=y_pred) * 100}%")